# [E-4] 작사가 인공지능 만들기
### 모재영(새싹 2기)

## I. 소스코드 

###### 1.  데이터 읽어오기

In [41]:
import os, re 
import numpy as np
import tensorflow as tf
import glob


txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me']


###### 2. 데이터 정제

In [42]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

[Hook]
I've been down so long, it look like up to me
They look up to me
I got fake people showin' fake love to me
Straight up to my face, straight up to my face
I've been down so long, it look like up to me
They look up to me
I got fake people showin' fake love to me
Straight up to my face, straight up to my face [Verse 1]
Somethin' ain't right when we talkin'


In [43]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [44]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> hook <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face verse <end>',
 '<start> somethin ain t right when we talkin <end>']

In [105]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words= 12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  953    3 ...    0    0    0]
 [   2    5   91 ...    0    0    0]
 [   2   45  134 ...    0    0    0]
 ...
 [   2  211    3 ...    0    0    0]
 [   2  399    9 ...    0    0    0]
 [   2    9 1294 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fdb1df6ed10>


In [106]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:155000, :14]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:155000, 1:15]    

print(len(src_input))
print(tgt_input[0])

155000
[953   3   0   0   0   0   0   0   0   0   0   0   0   0]


###### 3. 평가 데이터셋 분리

In [107]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                    tgt_input, 
                                                    test_size=0.2, 
                                                    random_state=10)

print('enc_train 개수: ', len(enc_train),', enc_val 개수: ', len(enc_val))

enc_train 개수:  124000 , enc_val 개수:  31000


- 당초 tensor[:, :14]경우 (140599, 15)로 나왔는데, tensor[:155000, :14]  경우로 수정해서 124000 으로 됨

In [108]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124000, 14)
Target Train: (124000, 14)


###### 4. 인공지능 만들기
- 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계

In [110]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 128 #embedding_size 는 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
hidden_size = 512 #hidden_size 는 모델에 얼마나 많은 일꾼을 둘 것인가
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [112]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train,dec_train, epochs=10)

Epoch 1/10
3875/3875 [==============================] - 174s 44ms/step - loss: 3.8097
Epoch 2/10
3875/3875 [==============================] - 142s 37ms/step - loss: 3.0935
Epoch 3/10
3875/3875 [==============================] - 141s 36ms/step - loss: 2.9002
Epoch 4/10
3875/3875 [==============================] - 141s 36ms/step - loss: 2.7515
Epoch 5/10
3875/3875 [==============================] - 142s 37ms/step - loss: 2.6411
Epoch 6/10
3875/3875 [==============================] - 143s 37ms/step - loss: 2.5345
Epoch 7/10
3875/3875 [==============================] - 142s 37ms/step - loss: 2.4523
Epoch 8/10
3875/3875 [==============================] - 142s 37ms/step - loss: 2.3698
Epoch 9/10
3875/3875 [==============================] - 142s 37ms/step - loss: 2.2957
Epoch 10/10
3875/3875 [==============================] - 142s 37ms/step - loss: 2.2334


- model.fit(x,y,batch_size=32,epochs=10)

In [114]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [121]:
generate_text(model, tokenizer, init_sentence="<start> you", max_len=20)

'<start> you know i m a motherfucking monster <end> '

## II. 회고

###### 1. 과정
 (1) 토큰을 잘라내서 소스 문장을 생성, 타겟 문장을 생성관련   
   
 당초 학습 데이터 개수가 140,599개 였다. 
 LMS상에 124,960보다 크다면 데이터 정제과정을 다시 한번 검토 하라는 점을 비추어 보면, 단순히 토큰화 했을 때의 토큰 개수가 15개 문장으로 국한시킨 것은 학습데이터개수와는 대별되는 것을 알게되었다. 이에 155,000로 토큰화할 문장으로 한정했다. 그 결과 124,960보다 적게 되었다.
   
 (2) Embedding Size와 Hidden Size를 조절
 당초 LMS상(셰익스피어)에 embedding_size는  256 / hidden_size는 1024 였다. 
 작사가 인공지능의 경우 단어가 추상적으로 표현되는 크기인 embedding_size는  128 / 모델에 얼마나 많은 일꾼을 둘 것인가 관련된 hidden_size는 512 경우 셰익스피어 보다 양은 많지만 덜 다양하다고 생각해서 적게잡았다
 
 (3) fit()함수에 대한 재조명
 LMS상(셰익스피어) fit()함수는 tf.data.Dataset.from_tensor_slices()메소드 때문에, 매개변수로 하나만 있으면 된다. 그러나 작사가 인공지능의 경우 train_test_split()메소드 때문에, 그 return 값으로 test,train 데이터가 분리돼서 나오기 때문에 fit()함수에는 매개변수로 모두 넣어주어야한다.


###### 2. 배운점
 (1) 이전에 했던 것들과 마찬가지로 자연어 처리도 데이터 전처리가 8할을 차지하는 것 같다. 정제함수로서 전처리에 문장부호, 대소문자, 특수문자 모두 반드시 챙겨가야한다.
 
 (2) 텐서와 관련된 배움
 첫번째 Tensor는 배열의 집합이다. / 두번째 차원의 수는 Rank와 같은말이다. / 세번째 배열의 차원에따라 불리는 이름이 달라진다.    
 즉 스칼라 -> 벡터 -> 메트릭스 -> 텐서 를 배웠다
 AI를 공부하면서 자주 출현하는 용어 인데 이번에 정립하고 서로 간의 관계를 알게됐다.


###### 3. 마치며

이렇게 자연어관련 자동으로 소설, 음악 가사 등을 얼마든지 인공지능으로 만들 수 있다면, 추후에 법관이나 변호사 등 법조계열을 위한 본안의 변론과정에서 필요한법 문서나 판결문을 자동으로 작성해주는 인공지능도 만들 수 있겠다고 생각한다.   
그 연유는 법은 법적안정성와 소송경제에 비추어보면 어느정도의 획일성이 필요한다.   
그만큼 예외발생가능성이 적기때문에 자연어 처리관련 분야중 가장 유효적절하게 운용할 수 있다고 생각한다. 